In this exercise we will play with a classical Boston Property Dataset.  It describes the prices of 
properties in Boston in '70. The task is to understand how they depend on multiple factors. Please 
consider the following URL for more details https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

    
Note the list of considered variables. Varianble number 14: MEDV - Median value of owner-occupied homes 
in $1000's is the descriptive variable we want to understand. 
In order to avoid extensve file processing, two files boston and boston_prices have been prepared for you. 
In the code below we read them and create the Ball Mapper plot.

This notebook was prepared by Davide Gurnari. 

In [ ]:
# uncomment and run this if you are on colab
# !apt install subversion
# !svn checkout https://github.com/dgurnari/pyBallMapper/trunk/src
# !svn checkout https://github.com/dgurnari/pyBallMapper/trunk/data

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
points_df = pd.read_csv('data/boston', sep='\t', 
                     header=None, names=['V'+str(i) for i in range(1,14)])
print(points_df.shape)
points_df.head()

In [ ]:
values_df = pd.read_csv('data/boston_prices', header=None, names=['price'])
print(values_df.shape)
values_df.head()

## Create Ball Mapper

In [ ]:
from src.pyBallMapper import BallMapper

In [ ]:
bm = BallMapper(points = points_df.values, # the pointcloud, as a numpy array
                epsilon = 100)             # the radius of the balls

In [ ]:
from matplotlib.colors import ListedColormap
from matplotlib import cm

my_rainbow_palette = cm.get_cmap(name='viridis')

In [ ]:
bm.add_coloring(values_df)
# we can color the graph by any column in coloring_df

plt.figure(figsize= (8,6))
bm.add_coloring(coloring_df=points_df)
# The BallMapper class has a builtin plotting method, buit around nx.draw_networkx
my_variable = 'price'
bm.draw_networx(coloring_variable=my_variable, color_palette=my_rainbow_palette, colorbar=True)
plt.title('BM graph colored by {}'.format(my_variable))
plt.show()

## More coloring

Since our dataset is relativelly low dimensional, we may use each of 13 attributes to see how it is distributed along the plot. Can you spot which variable seems to be most different among clusters?

In [ ]:
bm.add_coloring(points_df)
for variable in points_df.columns:
    # we can color the graph by any column in coloring_df
    plt.figure(figsize= (8,6))
    bm.add_coloring(coloring_df=points_df)
    bm.draw_networx(coloring_variable=variable, color_palette=my_rainbow_palette, colorbar=True)
    plt.title('BM graph colored by {}'.format(variable))
    plt.show()

## Comparing averages

Let us try to validate the observation from the previous point. By comparing the averages of two distributions (given by variables in the predefined regions of the graph) and returning the largest one.

In [ ]:
small = [10,11,12,13]
large = list(range(1,10))

In [ ]:
points_in_small = np.unique(np.concatenate([bm.points_covered_by_landmarks[node] 
                                            for node in small]))

In [ ]:
points_in_large = np.unique(np.concatenate([bm.points_covered_by_landmarks[node] 
                                            for node in large]))

In [ ]:
points_df.iloc[points_in_small].mean()

In [ ]:
points_df.iloc[points_in_large].mean()

In [ ]:
# absolute difference of the average, divided by the average in the whole dataset
(abs(points_df.iloc[points_in_small].mean() - points_df.iloc[points_in_large].mean()) \
/ points_df.mean()).sort_values(ascending=False)

This function somewhat confirms that the largest difference is obtained on the crime rate (variable 1), while the second largest on the index of accessibility to radial highway (communication indicator).